In [ ]:
# Mount Drive for saving models and data
from google.colab import drive
drive.mount('/content/drive')

# Install dependencies
!pip install streamlit transformers datasets torch accelerate sentencepiece evaluate rouge-score PyMuPDF pdfplumber langchain sentence-transformers

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
ERROR: Operation cancelled by user
^C


 # https://chatgpt.com/share/69009be4-fb98-800f-809e-32c5220ec092

In [ ]:
# c5444228a30d2426bbe734a52f94193fff57998f

In [ ]:
import os

base_dir = "/content/Auto_Insurance_Bot"
os.makedirs(f"{base_dir}/modules", exist_ok=True)
os.makedirs(f"{base_dir}/data", exist_ok=True)

print("✅ Folder structure created.")


✅ Folder structure created.


In [ ]:
import pandas as pd

data = [
    {
        "text": "This auto insurance policy provides coverage for third-party liability, theft, and collision damage up to ₹10,00,000. Excludes normal wear and tear, mechanical failure, and drunk driving incidents.",
        "summary": "Covers third-party, theft, and collision up to ₹10L. Excludes wear, mechanical damage, and drunk driving."
    },
    {
        "text": "The policy covers accidental damage to the vehicle, personal injury protection, and roadside assistance. Excludes pre-existing damage and unauthorized repairs.",
        "summary": "Provides accident, personal injury, and roadside assistance. Excludes pre-existing issues and unauthorized repairs."
    },
    {
        "text": "Includes comprehensive and third-party coverage with a deductible of ₹5,000. Premium ₹12,000/year. No coverage for natural calamities or riots.",
        "summary": "Comprehensive and third-party cover, ₹5k deductible, ₹12k premium. No coverage for natural calamities or riots."
    }
]

df = pd.DataFrame(data)
df.to_csv("/content/drive/MyDrive/auto_insurance_dataset.csv", index=False)
print("✅ Synthetic dataset saved at /content/drive/MyDrive/auto_insurance_dataset.csv")


✅ Synthetic dataset saved at /content/drive/MyDrive/auto_insurance_dataset.csv


In [ ]:
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, TrainingArguments, Trainer
import evaluate
import pandas as pd
import numpy as np # Import numpy

# Load dataset
df = pd.read_csv("/content/drive/MyDrive/auto_insurance_dataset.csv")
dataset = Dataset.from_pandas(df).train_test_split(test_size=0.1)

# Tokenization
model_name = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def preprocess(example):
    model_inputs = tokenizer(example["text"], truncation=True, padding="max_length", max_length=512)
    labels = tokenizer(example["summary"], truncation=True, padding="max_length", max_length=128)
    # Return labels as a list of lists
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized = dataset.map(preprocess, batched=True, remove_columns=dataset["train"].column_names)

# Load model
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
rouge = evaluate.load("rouge")

def compute_metrics(eval_pred):
    preds, labels = eval_pred
    # For sequence-to-sequence models, the predictions are usually logits.
    # We need to take the argmax to get the predicted token IDs.
    if isinstance(preds, tuple):
        preds = preds[0]
    preds = np.argmax(preds, axis=-1) # Take argmax to get token IDs

    # Convert numpy array of predictions to a list of lists
    decoded_preds = tokenizer.batch_decode(preds.tolist(), skip_special_tokens=True)

    # Decode labels - Trainer should handle -100
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    result = rouge.compute(predictions=decoded_preds, references=decoded_labels)
    return {k: round(v * 100, 2) for k, v in result.items()}


args = TrainingArguments(
    output_dir="/content/drive/MyDrive/aipa_model",
    eval_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=2,
    num_train_epochs=3,
    save_total_limit=1,
    logging_dir="/content/drive/MyDrive/logs",
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()

trainer.save_model("/content/drive/MyDrive/aipa_model")
print("✅ Fine-tuned model saved to Drive!")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Map:   0%|          | 0/2 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

/tmp/ipython-input-1331798373.py:55: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: Currently logged in as: shubham-dubey-btech2022 (shubham-dubey-btech2022-salesforce) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
1,No log,39.274651,17.140000,11.650000,17.140000,17.140000
2,No log,37.377766,17.140000,11.650000,17.140000,17.140000
3,No log,36.533245,17.140000,11.650000,17.140000,17.140000


✅ Fine-tuned model saved to Drive!


In [ ]:
from transformers import pipeline

summarizer = pipeline("summarization", model="/content/drive/MyDrive/aipa_model")

test_text = """
This auto insurance policy includes coverage for collision, theft, and natural disasters,
with a claim limit of ₹15,00,000. It excludes damage due to racing or commercial usage.
"""
summary = summarizer(test_text, max_length=120, min_length=40, do_sample=False)[0]['summary_text']
print("✅ Summary:\n", summary)


In [ ]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()


# Folder Creation cod e

In [ ]:
import os

# Create main project folder
os.makedirs("/content/Auto_Insurance_Bot", exist_ok=True)

# Move into it
os.chdir("/content/Auto_Insurance_Bot")

print("✅ Project folder created and opened:", os.getcwd())


✅ Project folder created and opened: /content/Auto_Insurance_Bot


In [ ]:
# Create all Python files
open("app.py", "w").close()
open("comparator.py", "w").close()
open("pdf_reader.py", "w").close()
open("summarizer.py", "w").close()
open("utils.py", "w").close()

print("✅ Files created:", os.listdir())


✅ Files created: ['summarizer.py', 'cloudflared-linux-amd64.deb.3', 'modules', '__pycache__', 'cloudflared-linux-amd64.deb.2', 'utils.py', 'pdf_reader.py', 'data', 'app.py', 'comparator.py', 'cloudflared-linux-amd64.deb.1', 'cloudflared-linux-amd64.deb', 'tunnel.log']


In [ ]:
!pip install -q streamlit transformers PyMuPDF
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i cloudflared-linux-amd64.deb
!pip install pyngrok
!pip install pyngrok
!apt install cloudflared


(Reading database ... 125085 files and directories currently installed.)
Preparing to unpack cloudflared-linux-amd64.deb ...
Unpacking cloudflared (2025.10.1) over (2025.10.1) ...
Setting up cloudflared (2025.10.1) ...
Processing triggers for man-db (2.10.2-1) ...
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
cloudflared is already the newest version (2025.10.1).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.


In [ ]:
import threading, time, subprocess

def run_streamlit():
    subprocess.run(["streamlit", "run", "app.py", "--server.port", "8501"])

# Start Streamlit in background
threading.Thread(target=run_streamlit, daemon=True).start()

# Wait for the app to start
time.sleep(8)

# Start Cloudflare tunnel
!cloudflared tunnel --url http://localhost:8501 --no-autoupdate > tunnel.log 2>&1 &

# Wait and fetch the public URL
time.sleep(5)
!grep -o "https://[-0-9a-z]*\.trycloudflare\.com" tunnel.log


https://southern-mtv-educators-mind.trycloudflare.com


# App

In [ ]:
# older


import streamlit as st
from utils import show_header
from pdf_reader import extract_text_from_pdf
from summarizer import summarize_policy
from comparator import compare_policies

show_header()

option = st.radio("Select Mode:", ["Summarize Policy", "Compare Quotes"])

if option == "Summarize Policy":
    file = st.file_uploader("Upload Auto Insurance Policy PDF", type=["pdf"])
    if file:
        text = extract_text_from_pdf(file)
        with st.spinner("Summarizing..."):
            summary = summarize_policy(text)
        st.subheader("🧾 Summary:")
        st.write(summary)

elif option == "Compare Quotes":
    files = st.file_uploader("Upload 2 or more Policy PDFs", type=["pdf"], accept_multiple_files=True)
    if len(files) >= 2:
        texts = [extract_text_from_pdf(f) for f in files]
        result = compare_policies(texts)
        st.subheader("📊 Comparison:")
        st.write(result)
    else:
        st.info("Please upload at least 2 policy PDFs.")


# comparator

In [ ]:
#older
def compare_policies(policies):
    summaries = []
    for i, text in enumerate(policies):
        summaries.append(f"Policy {i+1}: {len(text.split())} words long.")
    result = "\n".join(summaries)
    return "Comparison result:\n" + result


In [ ]:
from transformers import pipeline
import re

# Load summarizer
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

def clean_text(text):
    """Remove unwanted characters and compress whitespace."""
    text = re.sub(r'\s+', ' ', text)
    text = text.replace('\n', ' ').replace('\r', ' ')
    return text.strip()

def safe_summarize(text):
    """Summarize text safely."""
    text = clean_text(text)
    if not text or len(text.split()) < 30:
        return "Not enough content to summarize."

    # Truncate overly long text
    words = text.split()
    if len(words) > 800:
        text = " ".join(words[:800])

    try:
        output = summarizer(text, max_length=200, min_length=50, do_sample=False)
        if isinstance(output, list) and len(output) > 0 and "summary_text" in output[0]:
            return output[0]["summary_text"]
        else:
            return "Summary not available."
    except Exception as e:
        return f"Summarization failed: {str(e)}"

def compare_policies(policies):
    """Compare and recommend best auto insurance policy."""
    summaries = []
    for i, text in enumerate(policies):
        summary = safe_summarize(text)
        summaries.append({"policy": f"Policy {i+1}", "summary": summary})

    # Keyword scoring for auto insurance relevance
    keywords = ["coverage", "premium", "claim", "limit", "deductible", "damage", "vehicle", "theft"]
    scores = []
    for s in summaries:
        score = sum(s['summary'].lower().count(k) for k in keywords)
        scores.append(score)

    best_index = scores.index(max(scores)) if scores else 0

    # Construct a readable comparison output
    comparison_text = "📊 Auto Insurance Policy Comparison\n"
    for s, sc in zip(summaries, scores):
        comparison_text += f"\n{s['policy']}:\n{s['summary']}\n⭐ Keyword Relevance Score: {sc}\n"

    comparison_text += f"\n🏆 Recommendation: {summaries[best_index]['policy']} appears to offer stronger coverage and better benefits overall."
    return comparison_text


# Pdf Reader

In [ ]:
#older
import fitz  # PyMuPDF

def extract_text_from_pdf(file):
    text = ""
    with fitz.open(stream=file.read(), filetype="pdf") as doc:
        for page in doc:
            text += page.get_text()
    return text


In [ ]:
import fitz  # PyMuPDF

def extract_text_from_pdf(file):
    """
    Extract clean text from a PDF file using PyMuPDF.
    Handles multiple pages and removes extra spaces or line breaks.
    """
    text = ""

    # Open the PDF directly from uploaded file (stream)
    with fitz.open(stream=file.read(), filetype="pdf") as doc:
        for page_num, page in enumerate(doc, start=1):
            page_text = page.get_text("text").strip()
            text += f"\n--- Page {page_num} ---\n{page_text}\n"

    # Clean up unwanted whitespace and multiple newlines
    clean_text = " ".join(text.split())

    return clean_text


# summarizer

In [ ]:
from transformers import pipeline
import torch
import re

# ✅ Use GPU if available
device = 0 if torch.cuda.is_available() else -1
print("Using device:", "GPU (T4)" if device == 0 else "CPU")

# ✅ Use a lightweight summarization model for better speed + quality
summarizer = pipeline(
    "summarization",
    model="sshleifer/distilbart-cnn-12-6",
    device=device
)

def clean_text(text):
    """Clean input text by removing unnecessary symbols and spaces."""
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'[•●▪–-]+', '', text)
    text = text.replace("\n", " ").strip()
    return text

def bullet_point_format(text):
    """Convert summary text into neat bullet points."""
    # Split into sentences
    sentences = re.split(r'(?<=[.!?]) +', text)
    bullets = [f"• {s.strip()}" for s in sentences if len(s.strip()) > 10]
    return "\n".join(bullets)

def summarize_policy(text, target_word_count=250):
    """Summarize large policy text into clean 250-word bullet points."""
    text = clean_text(text)
    max_chunk = 1000  # safe chunk size
    sentences = re.split(r'(?<=[.!?]) +', text)
    current_chunk = ""
    chunks = []

    # Split text into chunks for summarization
    for sentence in sentences:
        if len(current_chunk) + len(sentence) <= max_chunk:
            current_chunk += " " + sentence
        else:
            chunks.append(current_chunk.strip())
            current_chunk = sentence
    if current_chunk:
        chunks.append(current_chunk.strip())

    print(f"🔹 Total chunks: {len(chunks)}")

    summaries = []
    for i, chunk in enumerate(chunks, 1):
        print(f"⏳ Summarizing chunk {i}/{len(chunks)} ...")
        summary = summarizer(
            chunk,
            max_length=150,
            min_length=60,
            do_sample=False
        )[0]["summary_text"]
        summaries.append(summary)

    combined_summary = " ".join(summaries)

    # Trim final summary to ~250 words
    words = combined_summary.split()
    if len(words) > target_word_count:
        combined_summary = " ".join(words[:target_word_count]) + "..."

    formatted_summary = bullet_point_format(combined_summary)
    return formatted_summary.strip()


# utils

In [ ]:
#older
import streamlit as st

def show_header():
    st.title("🚗 Auto Insurance Policy Assistant")
    st.subheader("Choose an option below:")


In [ ]:
import streamlit as st

def show_header():
    # App Title
    st.markdown(
        """
        <h1 style='text-align: center; color: #2E86C1;'>🚗 Auto Insurance Policy Assistant</h1>
        """,
        unsafe_allow_html=True
    )

    # Subtitle
    st.markdown(
        """
        <p style='text-align: center; font-size: 18px; color: #5D6D7E;'>
        Upload, summarize, and compare auto insurance policies in clear bullet points.
        </p>
        """,
        unsafe_allow_html=True
    )

    # Divider line for a clean layout
    st.markdown("<hr style='border: 1px solid #ccc;'>", unsafe_allow_html=True)

    # Option selector
    st.subheader("🔍 Choose an option below:")
    st.write(
        """
        - **📄 Summarize Policy:** Upload a PDF and get a clean, 250-word summary in bullet points.
        - **⚖️ Compare Policies:** Compare two policy summaries side-by-side to see which offers better coverage.
        - **💬 Insights:** Get key highlights, coverage, and exclusions automatically extracted.
        """
    )


# Api key c5444228a30d2426bbe734a52f94193fff57998f